In [2]:
# ===============================
# STEP 0: Install Libraries
# ===============================
!pip install -q youtube-transcript-api langchain langchain-community langchain-openai faiss-cpu tiktoken python-dotenv


In [6]:
# ===============================
# STEP 0.1: Import & API Key
# ===============================
import os

os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"  # 🔴 yaha apni key daalo


In [1]:
!pip uninstall -y youtube-transcript-api
!pip install --upgrade youtube-transcript-api


Found existing installation: youtube-transcript-api 1.2.3
Uninstalling youtube-transcript-api-1.2.3:
  Successfully uninstalled youtube-transcript-api-1.2.3
  Using cached youtube_transcript_api-1.2.3-py3-none-any.whl.metadata (24 kB)
Using cached youtube_transcript_api-1.2.3-py3-none-any.whl (485 kB)


In [3]:
!pip install pytube



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.7 MB/s eta 0:00:00


In [4]:
from pytube import YouTube

yt = YouTube("https://www.youtube.com/watch?v=ILrYwPd1Dc")

caption = yt.captions.get_by_language_code('en')
if caption:
    text = caption.generate_srt_captions()
    print(text[:1000])
else:
    print("❌ English captions available nahi hain")


RegexMatchError: regex_search: could not find match for (?:v=|\/)([0-9A-Za-z_-]{11}).*

In [ ]:
# ===============================
# STEP 2: Split Text into Chunks
# ===============================
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

documents = text_splitter.create_documents([transcript_text])

print(f"Total Chunks: {len(documents)}")
print(documents[0])


In [ ]:
# ===============================
# STEP 3: Create Embeddings & Store in FAISS
# ===============================
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

vector_store = FAISS.from_documents(documents, embeddings)


In [ ]:
# ===============================
# STEP 4: Create Retriever
# ===============================
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)

retriever


In [ ]:
# ===============================
# STEP 5: Prompt Template
# ===============================
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""
You are a helpful assistant.
Answer ONLY from the provided transcript context.
If the context is insufficient, say "I don't know".

Context:
{context}

Question:
{question}
""",
    input_variables=["context", "question"]
)


In [ ]:
# ===============================
# STEP 6: LLM
# ===============================
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.2
)


In [ ]:
# ===============================
# STEP 7: RAG Chain
# ===============================
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    RunnableParallel({
        "context": retriever | RunnableLambda(format_docs),
        "question": RunnablePassthrough()
    })
    | prompt
    | llm
    | StrOutputParser()
)


In [ ]:
# ===============================
# STEP 8: Ask Questions
# ===============================
rag_chain.invoke("Who is Demis Hassabis?")


In [ ]:
rag_chain.invoke("Is nuclear fusion discussed in this video?")


In [ ]:
rag_chain.invoke("Can you summarize the video?")


In [ ]:
YouTube Video
   ↓
Transcript
   ↓
Chunking
   ↓
Embeddings
   ↓
FAISS Vector DB
   ↓
Retriever
   ↓
Prompt Augmentation
   ↓
LLM
   ↓
Answer
